In [2]:
import numpy as np
import pandas as pd
import toytree
from scipy.optimize import minimize
from scipy.linalg import expm

In [48]:
def data_to_dict(data):
    """
    Parses data into format that can be used by the cond_like and
    pruningalg functions
    """
    values = [{0:-(i-1),1:i} for i in data]
    keys = list(range(0, len(data), 1))
    valuesdict = dict(zip(keys,values))
    return valuesdict

In [50]:
check = data_to_dict(data = testdata)
check

testtree = testtree.set_node_values('test', values = check)
testtree.get_node_values('test',True,True)

array(['', '', '', '', '', '', '', '', '', '', '', {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [63]:
def assign_tip_like_values(tree, data):
    """
    Assigns likelihood values to tree tips
    """
    values = [{0:-(i-1),1:i} for i in data]
    keys = list(range(0, len(data), 1))
    valuesdict = dict(zip(keys,values))
    tree = tree.set_node_values(feature = "likelihood", values = valuesdict)
    return tree

In [64]:
mytree = assign_tip_like_values(tree = testtree, data=testdata)
mytree.get_node_values('likelihood',True,True)

array(['', '', '', '', '', '', '', '', '', '', '', {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [19]:
def cond_like(likeleft0, likeleft1, likeright0, likeright1, tL, tR, alpha, beta):
    """
    Calculates conditional likelihood of character states at each node
    """

    Q = np.array([[-alpha, alpha], [beta, -beta]])
    probleft = expm(Q*tL)
    probright = expm(Q*tR)
 
    #ancestor is 0
    left0 = probleft[0, 0] * likeleft0 + probleft[0, 1] * likeleft1
    right0 = probright[0, 0] * likeright0 + probright[0, 1] * likeright1
    like_zero = left0*right0
 
    #ancestor is 1
    left1 = probleft[1, 0] * likeleft0 + probleft[1, 1] * likeleft1
    right1 = probright[1, 0] * likeright0 + probright[1, 1] * likeright1
    like_one = left1*right1
 
    return {0: like_zero, 1: like_one}

In [56]:
def pruningalg(tree, alpha, beta):
    """
    Runs Felsenstein's pruning algorithm on an input tree, given instantaneous transition
    rates alpha and beta. Assigns likelihood scores for characters states at each node.
    Specifically for binary state model. 
    """
    for node in tree.treenode.traverse("postorder"):
        if len(node.children) == 2:
            child1 = node.children[0]
            child2 = node.children[1]
            likedict = cond_like(likeright0 = child1.likelihood[0],
                                 likeright1 = child1.likelihood[1],
                                 likeleft0 = child2.likelihood[0],
                                 likeleft1 = child2.likelihood[1],
                                 tR = child1.dist,
                                 tL = child2.dist,
                                 alpha = alpha,
                                 beta = beta)
            node.likelihood = likedict

In [67]:
pruningalg(tree=mytree, alpha=8.0, beta=8.0)
mytree.get_node_values('likelihood',True,True)

array([{0: 0.00024414043275288296, 1: 0.00024414043275288304},
       {0: 0.03123932133513011, 1: 0.031239705219536906},
       {0: 0.007815164424494774, 1: 0.007815068389025239},
       {0: 0.24999997186620612, 1: 0.24999997186620618},
       {0: 0.12491614840992109, 1: 0.12499998593310307},
       {0: 0.12499998593310305, 1: 0.12491614840992114},
       {0: 0.06254191172814247, 1: 0.0625419117281425},
       {0: 0.24991613434302426, 1: 0.24991613434302432},
       {0: 0.24991613434302426, 1: 0.24991613434302432},
       {0: 0.2409260462126084, 1: 0.2592416851013426},
       {0: 0.2592416851013426, 1: 0.24092604621260852}, {0: 1, 1: 0},
       {0: 0, 1: 1}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 1, 1: 0}, {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 0, 1: 1},
       {0: 0, 1: 1}, {0: 1, 1: 0}, {0: 1, 1: 0}], dtype=object)

In [1]:
def full_lik2(x0, likeleft0, likeleft1, likeright0, likeright1, tL, tR, anca):
    """
    Get the likelihood of the data given the observed
    tip states (a, b) brlens (ta, tb) and ancestral
    state (anca) using two rate parameters (x0 array).
    """
    # get likelihood of ancestral states 0 or 1
    condlik = cond_lik2(a, b, ta, tb, x0[0], x0[1])
    
    # get full likelihoodlinke
    lik = (1 - anca) * node.likelihood[0] + (anca) * node.likelihood[1]
    
    # I don't understand this part
    if anca in [0., 1.]:
        lik /= 2
    
    return -lik #np.log(lik)

In [93]:
#according to Harmon, the likelihood of the whole tree is condlikeroot0*rootprior + condlikeroot0*rootprior
#so I want:
def node_like(tree):
    tree.set_node_values('neglike')
    for node in tree.treenode.traverse('postorder'):
        like = node.likelihood[0]*0.5 + node.likelihood[1]*0.5
        node.neglike = -like
    return tree

In [94]:
mytree2 = node_like(tree=mytree)
mytree2.get_node_values('neglike',True,True)

AttributeError: 'NoneType' object has no attribute 'get_node_values'

In [91]:
def model_fit(tree):
    """
    Find the maximum likelihood estimate of the two
    rate model parameters at each node given the data.
    """
    tree.set_node_values('alpha')
    tree.set_node_values('beta')
    for node in tree.treenode.traverse("postorder"):
        estimate = minimize(
            fun=node_like,
            x0=np.array([1., 1.]),
            method='L-BFGS-B',
            bounds=((0, 10), (0, 10))
            )
        result = {
            "alpha": round(estimate.x[0], 3),
            "beta": round(estimate.x[1], 3), 
            "convergence": estimate.success,
            }
        node.alpha = result['alpha']
        node.beta = result['beta']
    return tree

In [92]:
model_fit(tree=mytree)

AttributeError: 'numpy.ndarray' object has no attribute 'set_node_values'

In [24]:
testtree = toytree.rtree.unittree(ntips = 12)
testtree.draw(tree_style = 'p')

(<toyplot.canvas.Canvas at 0x7fd0dae9bf40>,
 <toytree.Render.ToytreeMark at 0x7fd0dab70640>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="265.0px" viewBox="0 0 350.0 265.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td9449a0fb105496b9a720ab653768195"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 0.0 0.5 1.0

In [25]:
testdata = [0,0,1,1,0,1,0,1,0,1,1,0]

In [47]:
testtree = testtree.set_node_values('num', values = {0:{1:0, 2:3}, 1:2, 2:3})
testtree.get_node_values('num', True, True)

array(['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',
       '', '', '', 3, 2, {1: 0, 2: 3}], dtype=object)